# Particle Swarm Optimization for motif finding (TFBS)
## first type of gaps

In [10]:
import numpy as np
import random
import math

class PSOMotifPlus:
  def __init__(self, sequences, motif_length, gap_length, num_agents, max_iterations, min_iterations, stagnation_threshold):
    self.sequences = sequences
    self.motif_length = motif_length
    self.gap_length = gap_length
    self.num_agents = num_agents
    self.max_iterations = max_iterations
    self.min_iterations = min_iterations
    self.stagnation_threshold = stagnation_threshold
    self.agent_initialization_strategy = 2
    self.num_bases = 4  # A, C, G, T
    self.agents = []
    self.pa = 0.25
    self.pc = 0.25
    self.pg = 0.25
    self.pt = 0.25
    self.background_frequency_list = [self.pa, self.pc, self.pg, self.pt]
    self.background_frequency = {"A": self.pa, "C": self.pc, "G": self.pg, "T": self.pt}
    self.base_dict = {"A": 0, "C": 1, "G": 2, "T": 3}

  def initialize_agents(self):
    if self.agent_initialization_strategy == 1:
      for _ in range(self.num_agents):
        agent = "".join(random.choices("ACGT", k=self.motif_length))
        self.agents.append(agent)
    else:
      for _ in range(self.num_agents):
        sequence = random.choice(self.sequences)
        upper_bound = len(sequence) - self.motif_length
        random_position = random.randrange(0, upper_bound + 1)
        agent = sequence[random_position:random_position + self.motif_length]
        self.agents.append(agent)

  def calculate_background_frequency(self):
    count = [0] * self.num_bases
    for sequence in self.sequences:
      for i in range(len(sequence)):
        current_base = sequence[i]
        count[self.base_dict[current_base]] += 1
    for n in range(self.num_bases):
      if count[n] == 0:
        count = [num + 1 for num in count]
    count_sum = sum(count[i] for i in range (len(count)))
    count = [count[i] / count_sum for i in range (len(count))]
    self.pa = count[0]
    self.pc = count[1]
    self.pg = count[2]
    self.pt = count[3]

  def calculate_matching_score(self, consensus, subsequence):
    score = 0
    for i in range(self.motif_length):
      xi = consensus[i]
      yi = subsequence[i]
      px = self.background_frequency[xi]
      py = self.background_frequency[yi]
      if xi == yi:
        score += 1 + math.log((0.25 / px), 4)
      else:
        score += math.log((0.25 / np.sqrt(px * py)), 4)
    return score

  def calculate_fitness(self, consensus):
    matches = []
    positions = []
    for sequence in self.sequences:
      best_score = float("-inf")
      best_match = None
      best_position = 0
      for i in range(len(sequence) - self.motif_length + 1):
        subsequence = sequence[i:i + self.motif_length]
        score = self.calculate_matching_score(consensus, subsequence)
        if score > best_score:
          best_score = score
          best_match = subsequence
          best_position = i
      matches.append(best_match)
      positions.append(best_position)
    ppm = self.calculate_ppm(matches)
    fitness, gapped_positions = self.calculate_information_content(ppm)
    return matches, positions, gapped_positions, fitness

  def calculate_ppm(self, matches):
    pfm = np.zeros((self.num_bases, self.motif_length))
    for match in matches:
      for i in range(self.motif_length):
        base = match[i]
        pfm[self.base_dict[base]][i] += 1
    ppm = pfm / len(matches)
    return ppm

  def calculate_information_content(self, ppm):
    information_content = 0
    information_content_list = [1.0]*self.motif_length
    for j in range(self.motif_length):
      column_Nfrequency = ppm[:, j]
      information_content_j = 0
      for b in range(self.num_bases):
        fb = column_Nfrequency[b]
        pb = self.background_frequency_list[b]
        if fb != 0:
          information_content_j += fb * np.log2(fb / pb)
        else:
          information_content_j += 0
      information_content_list[j] = information_content_j
    lowest_indices = sorted(range(len(information_content_list)), key=lambda i: information_content_list[i])[:self.gap_length]
    information_content_list.sort()
    information_content_list = information_content_list[self.gap_length:]
    information_content = sum(information_content_list)
    return information_content, lowest_indices

  def update_agents(self, pbest, gbest):
    for i in range(self.num_agents):
      agent = self.agents[i]
      pbest_agent = pbest[i]
      gbest_agent = gbest
      x = ["" for _ in range(4)]
      c = [1.0, 1.0, 1.0, 1.0]
      r = [0.25, 0.25, 0.25, 0.25]
      weights = [1.0, 1.0, 1.0, 1.0]
      weighted_values = [1.0, 1.0, 1.0, 1.0]
      for j in range(self.motif_length):
        x[0] = agent[j]
        x[1] = pbest_agent[j]
        x[2] = gbest_agent[j]
        x[3] = random.choice("ACGT")
    # relative importance of the four terms
        c[0] = 0.75
        c[1] = 1.0
        c[2] = 1.25
        c[3] = 0.5
        r[0] = random.uniform(0, 1)
        r[1] = random.uniform(0, 1)
        r[2] = random.uniform(0, 1)
        r[3] = random.uniform(0, 1)
        weights[0] = (1 - (self.background_frequency[x[0]])) / 3
        weights[1] = (1 - (self.background_frequency[x[1]])) / 3
        weights[2] = (1 - (self.background_frequency[x[2]])) / 3
        weights[3] = (1 - (self.background_frequency[x[3]])) / 3
        for k in range (self.num_bases):
          weighted_values[k] = c[k] * r[k] * weights[k]
        i_star = np.argmax(weighted_values)
        x_prime = x[i_star]
        agent_list = list(agent)
        agent_list[j] = x_prime
        temp_agent = ''.join(agent_list)
        agent = temp_agent

#################################################################################

  def psuedo_count(self, matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    for j in range(cols):
      has_zero = False
      for i in range(rows):
        if matrix[i][j] == 0:
          has_zero = True
          break
      if has_zero:
        for i in range(rows):
          matrix[i][j] += 1
    return matrix

  def column_sums(self, matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    sums = [0] * cols
    for j in range(cols):
      for i in range(rows):
        sums[j] += matrix[i][j]
    return sums

  def divide_columns(self, matrix, divisor):
      rows = len(matrix)
      cols = len(matrix[0])
      for j in range(cols):
          for i in range(rows):
              matrix[i][j] /= divisor[j]
      return matrix

  def calculate_ppm_for_pp(self, matches):
    pfm = np.zeros((self.num_bases, self.motif_length))
    for match in matches:
      for i in range(self.motif_length):
        base = match[i]
        pfm[self.base_dict[base]][i] += 1
    pfm = self.psuedo_count(pfm)
    col_sum = self.column_sums(pfm)
    ppm = self.divide_columns(pfm, col_sum)
    return ppm

  def post_processing(self, final_positions):
    found_motifs = []
    i = 0
    for seq in self.sequences:
      found_motifs.append(seq[final_positions[i]:(final_positions[i] + self.motif_length)])
      i += 1
    ppm = self.calculate_ppm_for_pp(found_motifs)
    scores = []
    subseq_scores = {}
    seq_num = 0
    for seq in self.sequences:
      for i in range(len(seq) - self.motif_length + 1):
        subseq = seq[i:i + self.motif_length]
        score = 0
        for pos in range(self.motif_length):
          base = subseq[pos]
          base_num = self.base_dict[base]
          prob = ppm[base_num][pos]
          background = self.background_frequency[base]
          score += np.log2(prob/ background)
        scores.append(score)
        subseq_scores[subseq] = []
        subseq_scores[subseq].append(score)
        subseq_scores[subseq].append(i)
        subseq_scores[subseq].append(seq_num)
      seq_num += 1
    min_sc = min(scores)
    max_sc = max(scores)
    real_motifs = {}
    for sub in subseq_scores:
      for i in range(0, len(subseq_scores[sub]), 3):
        score = subseq_scores[sub][i]
        if ((score - min_sc) / (max_sc - min_sc)) > 0.6:
          real_motifs[sub] = []
          real_motifs[sub].append(score)
          real_motifs[sub].append(subseq_scores[sub][i+1])
          real_motifs[sub].append(subseq_scores[sub][i+2])
    return real_motifs

######################################

  def optimize(self):
    self.calculate_background_frequency()
    final_consensus =  "".join(random.choices("ACGT", k=self.motif_length))
    final_consensus_fitness = -float("inf")
    final_consensus_positions = [0]*len(self.sequences)
    final_consensus_gapped_positions = [[0 for _ in range(self.gap_length)]for _ in range(len(self.sequences))]
    for reset in range(self.max_iterations):
      self.initialize_agents()
      pbest = self.agents.copy()
      pbest_fitness = [-float("inf") for _ in range(self.num_agents)]

      gbest = random.choice(self.agents)
      gbest_fitness = float("-inf")
      gbest_positions = [0]*len(self.sequences)
      gbest_gapped_positions = [[0 for _ in range(self.gap_length)]for _ in range(len(self.sequences))]
      stagnation_count = 0
      for iteration in range(self.min_iterations):
        prev_gbest = gbest
        for i in range(self.num_agents):
          consensus = self.agents[i]
          matches, start_positions, gapped_positions, fitness = self.calculate_fitness(consensus)
          if fitness > pbest_fitness[i]:
            pbest[i] = consensus
            pbest_fitness[i] = fitness
          if fitness > gbest_fitness:
            gbest = consensus
            gbest_fitness = fitness
            gbest_positions = start_positions
            gbest_gapped_positions = gapped_positions

        #check shift = built in the implementation due to fitness calculating function
        self.update_agents(pbest, gbest)
        if prev_gbest == gbest:
          stagnation_count += 1
        else:
          stagnation_count = 0
        if stagnation_count >= self.stagnation_threshold:
          break
      if gbest_fitness >= final_consensus_fitness:
        final_consensus = gbest
        final_consensus_fitness = gbest_fitness
        final_consensus_positions = gbest_positions
        final_consensus_gapped_positions = gbest_gapped_positions

    real_motifs = self.post_processing(final_consensus_positions)
    return final_consensus, final_consensus_fitness, final_consensus_positions, final_consensus_gapped_positions, real_motifs

In [6]:
# sequences = ["ATGCTAGCTAGCTAGC", "AGCTAGCTAGCTAGCT", "CTAGCTAGCTAGCTAG"]
# sequences = ["CCCCCATGCACCTTCGAAA", "AAAAAAAAAATGCGGG", "TTTATGCTTTTTTAAAAA"]
# sequences = ["TTCAGCGCAACGATTCCTGCTCCTTATGCAAGGGAAATCACCATACTGACCCCGCTTCTACTTACATGAGTCTGCAGACAGGCGTCTACCCCTCGCAGG",
#              "CCTCGGGTACAACTAGATGCACATATTAAAGCGAACCGAATAGCAAACCTTGACACCAACCGACTCTTATTATGACGTGCTTGCACTACACGATTGCCG",
#              "TAAGCTAGCGAGGCAAAATTATGAAAGGATTCCGATCACCAACGAAAGAGGGCCACTATGACTTCCACCTCTTGTTAGGACTCGTACCCCCATAGACAA"]
sequences = ["TAAGGCTAGCGAGG", "CAAAAAAAGGAT", "TCCGATCACCAAGG"]
motif_length = 4
gap_length = 2
num_agents = 30
max_iterations = 100
min_iterations = 10
stagnation_threshold = 20

pso_motif_plus = PSOMotifPlus(sequences, motif_length, gap_length, num_agents, max_iterations, min_iterations, stagnation_threshold)
final_consensus, final_consensus_fitness, final_consensus_positions, final_consensus_gapped_positions, real_motifs = pso_motif_plus.optimize()

found_motifs = []
i = 0
for seq in sequences:
  found_motifs.append(seq[final_consensus_positions[i]:(final_consensus_positions[i] + motif_length)])
  i += 1

gapped_found_motifs = []
for motif in found_motifs:
  for gpos in final_consensus_gapped_positions:
    motif_list = list(motif)
    motif_list[gpos] = 'n'
    motif = ''.join(motif_list)
  gapped_found_motifs.append(motif)

print("Optimization complete.")
print("Final consensus:", final_consensus)
print()
print("final consensus fitness: ", final_consensus_fitness)
print()
print("final consensus positions: ", final_consensus_positions)
print()
print("final consensus gap start position: ", final_consensus_gapped_positions, "    gap_length: ", gap_length)
print()
print("found motifs: ", found_motifs)
print()
print("found gapped motifs: ", gapped_found_motifs)
print()
print("real motifs: ", real_motifs)

Optimization complete.
Final consensus: GGAT

final consensus fitness:  4.0

final consensus positions:  [3, 8, 2]

final consensus gap start position:  [0, 2]     gap_length:  2

found motifs:  ['GGCT', 'GGAT', 'CGAT']

found gapped motifs:  ['nGnT', 'nGnT', 'nGnT']

real motifs:  {'GGCT': [3.3555428124907394, 3, 0], 'CGAG': [1.3555428124907394, 9, 0], 'GGAT': [3.9405053132118955, 8, 1], 'CGAT': [3.3555428124907394, 2, 2]}


## CRP

In [11]:
CRP_sequences = ["TAATGTTTGTGCTGGTTTTTGTGGCATCGGGCGAGAATAGCGCGTGGTGTGAAAGACTGTTTTTTTGATCGTTTTCACAAAAATGGAAGTCCACAGTCTTGACAG",
                  "GACAAAAACGCGTAACAAAAGTGTCTATAATCACGGCAGAAAAGTCCACATTGATTATTTGCACGGCGTCACACTTTGCTATGCCATAGCATTTTTATCCATAAG",
                  "ACAAATCCCAATAACTTAATTATTGGGATTTGTTATATATAACTTTATAAATTCCTAAAATTACACAAAGTTAATAACTGTGAGCATGGTCATATTTTTATCAAT",
                  "CACAAAGCGAAAGCTATGCTAAAACAGTCAGGATGCTACAGTAATACATTGATGTACTGCATGTATGCAAAGGACGTCACATTACCGTGCAGTACAGTTGATAGC",
                  "ACGGTGCTACACTTGTATGTAGCGCATCTTTCTTTACGGTCAATCAGCATGGTGTTAAATTGATCACGTTTTAGACCATTTTTTCGTCGTGAAACTAAAAAAACC",
                  "AGTGAATTATTTGAACCAGATCGCATTACAGTGATGCAAACTTGTAAGTAGATTTCCTTAATTGTGATGTGTATCGAAGTGTGTTGCGGAGTAGATGTTAGAATA",

                  "GCGCATAAAAAACGGCTAAATTCTTGTGTAAACGATTCCACTAATTTATTCCATGTCACACTTTTCGCATCTTTGTTATGCTATGGTTATTTCATACCATAAGCC",
                  "GCTCCGGCGGGGTTTTTTGTTATCTGCAATTCAGTACAAAACGTGATCAACCCCTCAATTTTCCCTTTGCTGAAAAATTTTCCATTGTCTCCCCTGTAAAGCTGT",
                  "AACGCAATTAATGTGAGTTAGCTCACTCATTAGGCACCCCAGGCTTTACACTTTATGCTTCCGGCTCGTATGTTGTGTGGAATTGTGAGCGGATAACAATTTCAC",
                  "ACATTACCGCCAATTCTGTAACAGAGATCACACAAAGCGACGGTGGGGCGTAGGGGCAAGGAGGATGGAAAGAGGTTGCCGTATAAAGAAACTAGAGTCCGTTTA",
                  "GGAGGAGGCGGGAGGATGAGAACACGGCTTCTGTGAACTAAACCGAGGTCATGTAAGGAATTTCGTGATGTTGCTTGCAAAAATCGTGGCGATTTTATGTGCGCA",
                  "GATCAGCGTCGTTTTAGGTGAGTTGTTAATAAAGATTTGGAATTGTGACACAGTGCAAATTCAGACACATAAAAAAACGTCATCGCTTGCATTAGAAAGGTTTCT",

                  "GCTGACAAAAAAGATTAAACATACCTTATACAAGACTTTTTTTTCATATGCCTGACGGAGTTCACACTTGTAAGTTTTCAACTACGTTGTAGACTTTACATCGCC",
                  "TTTTTTAAACATTAAAATTCTTACGTAATTTATAATCTTTAAAAAAAGCATTTAATATTGCTCCCCGAACGATTGTGATTCGATTCACATTTAAACAATTTCAGA",
                  "CCCATGAGAGTGAAATTGTTGTGATGTGGTTAACCCAATTAGAATTCGGGATTGACATGTCTTACCAAAAGGTAGAACTTATACGCCATCTCATCCGATGCAAGC",
                  "CTGGCTTAACTATGCGGCATCAGAGCAGATTGTACTGAGAGTGCACCATATGCGGTGTGAAATACCGCACAGATGCGTAAGGAGAAAATACCGCATCAGGCGCTC",
                  "CTGTGACGGAAGATCACTTCGCAGAATAAATAAATCCTGGTGTCCCTGTTGATACCGGGAAGCCCTGGGCCAACTTTTGGCGAAAATGAGACGTTGATCGGCACG",
                  "GATTTTTATACTTTAACTTGTTGATATTTAAAGGTATTTAATTGTAATAACGATACTCTGGAAAGTATTGAAAGTTAATTTGTGAGTGGTCGCACATATCCTGTT"]

motif_length = 20
gap_length = 4
num_agents = 30
max_iterations = 100
min_iterations = 10
stagnation_threshold = 20

pso_motif_plus = PSOMotifPlus(CRP_sequences, motif_length, gap_length, num_agents, max_iterations, min_iterations, stagnation_threshold)
final_consensus_CRP, final_consensus_fitness_CRP, final_consensus_positions_CRP, final_consensus_gapped_positions_CRP, real_motifs_CRP = pso_motif_plus.optimize()

found_motifs_CRP = []
i = 0
for seq in CRP_sequences:
  found_motifs_CRP.append(seq[final_consensus_positions_CRP[i]:(final_consensus_positions_CRP[i] + motif_length)])
  i += 1

gapped_found_motifs_CRP = []
for motif in found_motifs_CRP:
  for gpos in final_consensus_gapped_positions_CRP:
    motif_list = list(motif)
    motif_list[gpos] = 'n'
    motif = ''.join(motif_list)
  gapped_found_motifs_CRP.append(motif)

print("Optimization complete.")
print("Final consensus:", final_consensus_CRP)
print()
print("final consensus fitness: ", final_consensus_fitness_CRP)
print()
print("final consensus positions: ", final_consensus_positions_CRP)
print()
print("final consensus gap start position: ", final_consensus_gapped_positions_CRP, "    gap_length: ", gap_length)
print()
print("found motifs: ", found_motifs_CRP)
print()
print("found gapped motifs: ", gapped_found_motifs_CRP)
print()
print("real motifs: ", real_motifs_CRP)

Optimization complete.
Final consensus: ATTCAGACACATAAAAAAAC

final consensus fitness:  9.313060401591084

final consensus positions:  [72, 1, 50, 5, 84, 33, 3, 22, 18, 72, 5, 58, 0, 27, 50, 72, 15, 36]

final consensus gap start position:  [5, 9, 3, 1]     gap_length:  4

found motifs:  ['TTTCACAAAAATGGAAGTCC', 'ACAAAAACGCGTAACAAAAG', 'ATTCCTAAAATTACACAAAG', 'AGCGAAAGCTATGCTAAAAC', 'CGTCGTGAAACTAAAAAAAC', 'ATGCAAACTTGTAAGTAGAT', 'CATAAAAAACGGCTAAATTC', 'TCTGCAATTCAGTACAAAAC', 'TAGCTCACTCATTAGGCACC', 'AGGTTGCCGTATAAAGAAAC', 'AGGCGGGAGGATGAGAACAC', 'ATTCAGACACATAAAAAAAC', 'GCTGACAAAAAAGATTAAAC', 'ATTTATAATCTTTAAAAAAA', 'ATTGACATGTCTTACCAAAA', 'ATGCGTAAGGAGAAAATACC', 'ACTTCGCAGAATAAATAAAT', 'TTTAATTGTAATAACGATAC']

found gapped motifs:  ['TnTnAnAAAnATGGAAGTCC', 'AnAnAnACGnGTAACAAAAG', 'AnTnCnAAAnTTACACAAAG', 'AnCnAnAGCnATGCTAAAAC', 'CnTnGnGAAnCTAAAAAAAC', 'AnGnAnACTnGTAAGTAGAT', 'CnTnAnAAAnGGCTAAATTC', 'TnTnCnATTnAGTACAAAAC', 'TnGnTnACTnATTAGGCACC', 'AnGnTnCCGnATAAAGAAAC', 'AnGnGnGAGnATG

## Avarage Performance Measure Functions

In [30]:
def precision(c, p):
  return c / p

def recall(c, t):
  return c / t

def F_score(precision, recall):
  return (2 * precision * recall) / (precision + recall)

def performance_measure(c, p, t):
  prec = precision(c, p)
  rec = recall(c, t)
  F_s = F_score(prec, rec)
  return [prec, rec, F_s]

In [23]:
import pandas as pd
df_CRP = pd.DataFrame(real_motifs_CRP)
df_CRP

,TTTGATCGTTTTCACAAAAA,TTTCACAAAAATGGAAGTCC,ACAAAAACGCGTAACAAAAG,CGTAACAAAAGTGTCTATAA,ATGCCATAGCATTTTTATCC,TTTGTTATATATAACTTTAT,ATATATAACTTTATAAATTC,TTATAAATTCCTAAAATTAC,AATTCCTAAAATTACACAAA,ATTCCTAAAATTACACAAAG,...,TTTATAATCTTTAAAAAAAG,CTTTAAAAAAAGCATTTAAT,ATTCACATTTAAACAATTTC,GGTTAACCCAATTAGAATTC,ATTGACATGTCTTACCAAAA,ATGCGTAAGGAGAAAATACC,ACTTCGCAGAATAAATAAAT,CTTCGCAGAATAAATAAATC,TTTAATTGTAATAACGATAC,ACTCTGGAAAGTATTGAAAG
0,4.998097,7.872236,10.208312,2.904658,3.078687,3.509666,3.226586,5.311475,4.554061,10.649492,...,6.979493,3.210106,4.576781,2.775011,8.777147,10.380031,11.063836,5.428746,8.529898,4.042161
1,63.000000,72.000000,1.000000,10.000000,80.000000,28.000000,34.000000,44.000000,49.000000,50.000000,...,28.000000,36.000000,82.000000,27.000000,50.000000,72.000000,15.000000,16.000000,36.000000,54.000000
2,0.000000,0.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,13.000000,13.000000,13.000000,14.000000,14.000000,15.000000,16.000000,16.000000,17.000000,17.000000


In [21]:
def filter_df(df):
  scores = df.iloc[0, 1:]
  q1 = np.percentile(scores, 25)
  q2 = np.percentile(scores, 50)
  q3 = np.percentile(scores, 75)
  iqr = q3 - q1
  cut_off = q1 = iqr
  filtered_df = df.loc[:, df.columns[1:][scores >= cut_off]]
  return filtered_df

In [33]:
filtered_df_CRP = filter_df(df_CRP)
filtered_df_CRP

,TTTCACAAAAATGGAAGTCC,ACAAAAACGCGTAACAAAAG,TTATAAATTCCTAAAATTAC,ATTCCTAAAATTACACAAAG,AATTACACAAAGTTAATAAC,ATTACACAAAGTTAATAACT,AGCGAAAGCTATGCTAAAAC,CGTCGTGAAACTAAAAAAAC,ATGCAAACTTGTAAGTAGAT,TCTGCAATTCAGTACAAAAC,...,ACAAAAAAGATTAAACATAC,ACTTGTAAGTTTTCAACTAC,TTTTTTAAACATTAAAATTC,ATTTATAATCTTTAAAAAAA,TTTATAATCTTTAAAAAAAG,ATTGACATGTCTTACCAAAA,ATGCGTAAGGAGAAAATACC,ACTTCGCAGAATAAATAAAT,CTTCGCAGAATAAATAAATC,TTTAATTGTAATAACGATAC
0,7.872236,10.208312,5.311475,10.649492,7.184111,7.434479,5.613648,12.203866,8.244838,10.684037,...,9.788828,5.36409,11.171445,14.667744,6.979493,8.777147,10.380031,11.063836,5.428746,8.529898
1,72.000000,1.000000,44.000000,50.000000,58.000000,59.000000,5.000000,84.000000,33.000000,22.000000,...,4.000000,65.00000,0.000000,27.000000,28.000000,50.000000,72.000000,15.000000,16.000000,36.000000
2,0.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,5.000000,7.000000,...,12.000000,12.00000,13.000000,13.000000,13.000000,14.000000,15.000000,16.000000,16.000000,17.000000


In [34]:
c_CRP = 25
p_CRP = 25
t_CRP = 37

In [35]:
CRP_accuracy = performance_measure(c_CRP, p_CRP, t_CRP)
CRP_accuracy

[1.0, 0.6756756756756757, 0.8064516129032258]